In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import time
import copy
import warnings
warnings.filterwarnings('ignore', message="Objective thresholds were not set")
warnings.filterwarnings('ignore', message="`is_ordered` is not specified")
warnings.filterwarnings('ignore', message="`AxParameterWarning")
sys.path.insert(0, '/root/sxw/edit')

from hybrid_optimizer import HybridOptimizer
from ax_optimizer import ExperimentResult
from LLINBO_agent import (
    ProblemContext, Parameter, PriorExperiment, 
    LLMConfig
)

# 设置环境变量
os.environ.setdefault("OPENAI_API_KEY", "your_openai_api_key")
os.environ.setdefault("HTTP_PROXY", "http://127.0.0.1:7890")
os.environ.setdefault("HTTPS_PROXY", "http://127.0.0.1:7890")

print("✅ 库导入完成")

✅ 库导入完成


In [2]:
def load_chemical_data(csv_path: str):
    """加载化学实验数据"""
    df = pd.read_csv(csv_path)
    return df


def build_llm_parameter_space(df: pd.DataFrame):
    """构建 LLM 参数空间定义（Parameter 格式）"""
    parameters = [
        Parameter(
            name="base",
            type="choice",
            values=sorted(df["base"].unique().tolist()),
            value_type="str",
            description="碱基类型",
        ),
        Parameter(
            name="ligand",
            type="choice",
            values=sorted(df["ligand"].unique().tolist()),
            value_type="str",
            description="配体类型",
        ),
        Parameter(
            name="solvent",
            type="choice",
            values=sorted(df["solvent"].unique().tolist()),
            value_type="str",
            description="溶剂类型",
        ),
        Parameter(
            name="concentration",
            type="choice",
            values=sorted(df["concentration"].unique().tolist()),
            value_type="float",
            description="反应浓度 (M)",
        ),
        Parameter(
            name="temperature",
            type="choice",
            values=sorted(df["temperature"].unique().tolist()),
            value_type="int",
            description="反应温度 (°C)",
        ),
    ]
    return parameters


def build_gp_parameter_space(df: pd.DataFrame):
    """构建 GP 参数空间定义（Ax 格式）"""
    parameters = [
        {
            "name": "base",
            "type": "choice",
            "values": sorted(df["base"].unique().tolist()),
            "value_type": "str",
        },
        {
            "name": "ligand",
            "type": "choice",
            "values": sorted(df["ligand"].unique().tolist()),
            "value_type": "str",
        },
        {
            "name": "solvent",
            "type": "choice",
            "values": sorted(df["solvent"].unique().tolist()),
            "value_type": "str",
        },
        {
            "name": "concentration",
            "type": "choice",
            "values": sorted(df["concentration"].unique().tolist()),
            "value_type": "float",
        },
        {
            "name": "temperature",
            "type": "choice",
            "values": sorted(df["temperature"].unique().tolist()),
            "value_type": "int",
        },
    ]
    return parameters


def convert_parameters_to_api_format(parameters):
    """将 LLINBO Parameter 转换为 API ParameterSpace 格式"""
    api_params = []
    for param in parameters:
        api_param = {
            "name": param.name,
            "type": param.type,
            "values": param.values
        }
        api_params.append(api_param)
    return api_params


def call_init_api(parameter_space, objectives, batch_size=20, seed=None, api_url="http://localhost:3320"):
    """调用 init 接口进行采样"""
    init_endpoint = f"{api_url}/init"
    
    # 转换参数空间格式
    api_parameter_space = convert_parameters_to_api_format(parameter_space)
    
    # 构建请求数据
    init_request = {
        "parameter_space": api_parameter_space,
        "objectives": list(objectives.keys()),
        "batch": batch_size,
        "seed": seed,
        "sampling_method": "lhs"  # 使用 lhs 采样
    }
    
    print(f"🚀 调用 init 接口，批次大小: {batch_size}")
    
    try:
        response = requests.post(init_endpoint, json=init_request, timeout=30)
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Init 接口调用成功")
            print(f"📈 生成参数组合数: {len(result['results'])}")
            print(f"💬 消息: {result['message']}")
            return result
        else:
            print(f"❌ Init 接口调用失败: {response.status_code}")
            print(f"📄 错误信息: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Init 接口调用异常: {str(e)}")
        return None


def simulate_experiment_results(params_list, df, random_seed=None):
    """从真实数据中查找完全匹配的实验结果"""
    if random_seed is not None:
        np.random.seed(random_seed)
    
    results = []
    
    for params in params_list:
        # 构建精确匹配的查询条件
        query_conditions = []
        for key, value in params.items():
            if isinstance(value, str):
                query_conditions.append(f"{key} == '{value}'")
            else:
                query_conditions.append(f"{key} == {value}")
        
        # 构建查询字符串
        query_str = " and ".join(query_conditions)
        
        try:
            # 查找完全匹配的数据
            matched_data = df.query(query_str)
            
            if len(matched_data) > 0:
                # 取第一个匹配的结果
                row = matched_data.iloc[0]
                experiment_result = {
                    "parameters": params,
                    "metrics": {
                        "yield": float(row['yield']),
                        "cost": float(row['cost'])
                    }
                }
                results.append(experiment_result)
                print(f"✅ 找到完全匹配: {params} -> yield={row['yield']:.2f}, cost={row['cost']:.3f}")
            else:
                print(f"⚠️ 未找到完全匹配: {params}")
                # 如果找不到，使用随机结果
                random_row = df.sample(1, random_state=random_seed).iloc[0]
                experiment_result = {
                    "parameters": params,
                    "metrics": {
                        "yield": float(random_row['yield']),
                        "cost": float(random_row['cost'])
                    }
                }
                results.append(experiment_result)
                print(f"⚠️ 使用随机结果: {params} -> yield={random_row['yield']:.2f}, cost={random_row['cost']:.3f}")
                
        except Exception as e:
            print(f"❌ 查询失败: {str(e)}")
            # 使用随机结果
            random_row = df.sample(1, random_state=random_seed).iloc[0]
            experiment_result = {
                "parameters": params,
                "metrics": {
                    "yield": float(random_row['yield']),
                    "cost": float(random_row['cost'])
                }
            }
            results.append(experiment_result)
    
    return results


def convert_experiment_results_to_experiment_results(experiment_results):
    """将实验结果转换为 ExperimentResult 格式"""
    results = []
    for result in experiment_results:
        exp = ExperimentResult(
            parameters=result["parameters"],
            metrics=result["metrics"]
        )
        results.append(exp)
    return results


def create_prior_experiments(df: pd.DataFrame, parameters, objectives, n_samples: int = 50, 
                             api_url: str = "http://localhost:3320", seed: int = None):
    """使用 init 接口进行采样，创建先验实验数据"""
    print(f"\n📚 使用 init 接口进行采样，生成 {n_samples} 个先验实验数据...")
    
    # 调用 init 接口进行采样
    init_result = call_init_api(parameters, objectives, batch_size=n_samples, seed=seed, api_url=api_url)
    
    if not init_result:
        print("⚠️ Init 接口调用失败，使用随机采样作为备选方案")
        # 备选方案：随机采样
        sample_df = df.sample(n=min(n_samples, len(df)), random_state=seed)
        prior_experiments = []
        for _, row in sample_df.iterrows():
            exp = ExperimentResult(
                parameters={
                    "base": str(row["base"]),
                    "ligand": str(row["ligand"]),
                    "solvent": str(row["solvent"]),
                    "concentration": float(row["concentration"]),
                    "temperature": int(row["temperature"]),
                },
                metrics={
                    "yield": float(row["yield"]),
                    "cost": float(row["cost"]),
                }
            )
            prior_experiments.append(exp)
        return prior_experiments
    
    # 从数据集中查找匹配的实验结果
    print(f"\n🔍 从数据集中查找匹配的实验结果...")
    experiment_results = simulate_experiment_results(init_result['results'], df, random_seed=seed)
    
    # 转换为 ExperimentResult 格式
    prior_experiments = convert_experiment_results_to_experiment_results(experiment_results)
    
    print(f"✅ 成功创建 {len(prior_experiments)} 个先验实验数据")
    
    return prior_experiments

print("✅ 辅助函数定义完成")

✅ 辅助函数定义完成


In [3]:
csv_path = "test/1728_BMS_experiments_yield_and_cost.csv"
df = load_chemical_data(csv_path)
print("\n🔧 构建参数空间...")
llm_parameters = build_llm_parameter_space(df)
gp_search_space = build_gp_parameter_space(df)
print(f"✅ LLM 参数空间定义完成: {len(llm_parameters)} 个参数")
print(f"✅ GP 参数空间定义完成: {len(gp_search_space)} 个参数")
for param in llm_parameters:
    print(f"   - {param.name}: {len(param.values)} 个可选值")
problem_context = ProblemContext(
    problem_description="优化 Pd 催化的 C-H 芳基化反应条件，反应物为 N1-甲基-1H-咪唑-4-甲腈和 1-溴-2-氟苯",
    industry="化学合成 - 有机反应优化",
    domain_knowledge="""
    这是一个 Pd 催化的 C-H 芳基化反应优化问题。
    - 碱基（base）影响反应活性和选择性
    - 配体（ligand）对催化剂活性和选择性至关重要
    - 溶剂（solvent）影响反应速率和产物选择性
    - 浓度（concentration）影响反应速率和副反应
    - 温度（temperature）影响反应速率和选择性
    优化目标是同时最大化产率（yield）和最小化成本（cost）。
    """,
    constraints=[
        "所有推荐的参数组合必须符合基本化学常识"
    ],
    optimization_goals=[
        "最大化产率（yield）",
        "最小化成本（cost）"
    ]
)
print("✅ 问题背景定义完成")


🔧 构建参数空间...
✅ LLM 参数空间定义完成: 5 个参数
✅ GP 参数空间定义完成: 5 个参数
   - base: 4 个可选值
   - ligand: 12 个可选值
   - solvent: 4 个可选值
   - concentration: 3 个可选值
   - temperature: 3 个可选值
✅ 问题背景定义完成


In [4]:
objectives = {
    "yield": {"minimize": False},  # 最大化产率
    "cost": {"minimize": True}      # 最小化成本
}
optimization_config = {
    "objectives": objectives
}
print("✅ 优化目标定义完成")
print(f"   - yield: 最大化")
print(f"   - cost: 最小化")

✅ 优化目标定义完成
   - yield: 最大化
   - cost: 最小化


In [5]:
print("\n📚 创建先验实验数据...")
prior_experiments = create_prior_experiments(
    df, llm_parameters, objectives, 
    n_samples=10,  # 初始批次大小
    api_url="http://localhost:3320",
    seed=42
)
print(f"✅ 先验实验数据: {len(prior_experiments)} 个实验")


📚 创建先验实验数据...

📚 使用 init 接口进行采样，生成 10 个先验实验数据...
🚀 调用 init 接口，批次大小: 10
✅ Init 接口调用成功
📈 生成参数组合数: 10
💬 消息: 初始化成功，使用lhs采样生成10个参数组合

🔍 从数据集中查找匹配的实验结果...
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PPh3', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 90} -> yield=5.80, cost=0.090
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'BrettPhos', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 90} -> yield=28.02, cost=0.199
✅ 找到完全匹配: {'base': 'CsOPiv', 'ligand': 'GorlosPhos HBF4', 'solvent': 'p-Xylene', 'concentration': 0.153, 'temperature': 105} -> yield=4.53, cost=0.122
✅ 找到完全匹配: {'base': 'KOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.057, 'temperature': 120} -> yield=77.58, cost=0.045
✅ 找到完全匹配: {'base': 'KOPiv', 'ligand': 'P(fur)3', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 120} -> yield=53.11, cost=0.045
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PCy3 HBF4', 'solvent': 'BuOAc', 'concentration': 0.057, 'temperature': 105} -> yield=0.00, cost=0.108
✅ 找到完全匹配: 

In [6]:
print("\n🤖 初始化 Hybrid Optimizer...")
llm_config = LLMConfig(
    model_name="gpt-4.1-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.openai.com/v1"
)

hybrid_optimizer = HybridOptimizer(
    llm_parameters=llm_parameters,
    gp_search_space=gp_search_space,
    optimization_config=optimization_config,
    problem_context=problem_context,
    llm_config=llm_config,
    acquisition_threshold=0.1,  # 采集函数差值阈值
    random_seed=42,
    gp_random_seed=42
)


🤖 初始化 Hybrid Optimizer...


[WARNING 11-14 02:10:24] ax.service.ax_client: Random seed set to 42. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[WARNING 11-14 02:10:24] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.


🚀 使用GPU加速: NVIDIA A100-SXM4-80GB


/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "base". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "ligand". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "solvent". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior

In [7]:
# 添加先验数据
hybrid_optimizer.add_prior_experiments(prior_experiments)
print("✅ Hybrid Optimizer 初始化完成")

[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 0 with data: {'yield': (5.8, 0.0), 'cost': (0.089659, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 1 with data: {'yield': (28.02, 0.0), 'cost': (0.198802, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 2 with data: {'yield': (4.53, 0.0), 'cost': (0.121718, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 3 with data: {'yield': (77.58, 0.0), 'cost': (0.04484, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 4 with data: {'yield': (53.11, 0.0), 'cost': (0.045232, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 5 with data: {'yield': (0.0, 0.0), 'cost': (0.107935, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 6 with data: {'yield': (24.0, 0.0), 'cost': (0.155989, 0.0)}.
[INFO 11-14 02:10:24] ax.service.ax_client: Completed trial 7 with data: {'yield': (24.02, 0.0), 'cost': (0.264828, 0.0)}.
[INFO 11-14 02:10:24] a

✅ Hybrid Optimizer 初始化完成


In [8]:
print("\n🎯 生成融合优化建议...")
num_suggestions = 5
suggestions = hybrid_optimizer.suggest_parameters(
    num_suggestions=num_suggestions,
    use_llm=True,
    use_gp=True,
    print_details=True
)
print(f"✅ 成功生成 {len(suggestions)} 个融合建议")


🎯 生成融合优化建议...


/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/torch/random.py:187: UserWarning:

CUDA reports that you have 8 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in t


📊 GP 推荐最佳点:
   参数: {'concentration': 0.1, 'temperature': 120, 'base': 'KOPiv', 'ligand': 'X-Phos', 'solvent': 'DMAc'}
   采集函数值: -0.435627
   预测标准差: 0.555671

📝 输入大模型的完整提示词:
# 优化问题背景
**问题描述**: 优化 Pd 催化的 C-H 芳基化反应条件，反应物为 N1-甲基-1H-咪唑-4-甲腈和 1-溴-2-氟苯
**行业领域**: 化学合成 - 有机反应优化
**领域知识**: 
    这是一个 Pd 催化的 C-H 芳基化反应优化问题。
    - 碱基（base）影响反应活性和选择性
    - 配体（ligand）对催化剂活性和选择性至关重要
    - 溶剂（solvent）影响反应速率和产物选择性
    - 浓度（concentration）影响反应速率和副反应
    - 温度（temperature）影响反应速率和选择性
    优化目标是同时最大化产率（yield）和最小化成本（cost）。
    
**约束条件**: 所有推荐的参数组合必须符合基本化学常识
**优化目标**: 最大化产率（yield）, 最小化成本（cost）

# 参数空间定义
以下是需要优化的参数及其范围：

**重要提示**：如果参数是离散参数，必须从列出的可选值中选择，不能选择其他值；如果参数是连续参数，值必须在 [最小值, 最大值] 范围内。
1. **base**
   描述: 碱基类型
   类型: 离散参数（必须从以下值中选择一个）
   可选值列表: [CsOAc, CsOPiv, KOAc, KOPiv]
   可选值数量: 4 个
   ⚠️ 重要：只能选择列表中的值，不能选择其他值或中间值
   值类型: str
2. **ligand**
   描述: 配体类型
   类型: 离散参数（必须从以下值中选择一个）
   可选值列表: [BrettPhos, CgMe-PPh, GorlosPhos HBF4, JackiePhos, P(fur)3, PCy3 HBF4, PPh2Me, PPh3, PPhMe2, PPhtBu2, X-Phos, tBPh-CPhos]
 

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3


   LLM 推荐点: {'base': 'KOAc', 'ligand': 'P(fur)3', 'solvent': 'p-Xylene', 'concentration': 0.1, 'temperature': 105}
   采集函数值: -1.604131
   预测标准差: 0.554274
   与最佳点差值: 1.168504
   动态阈值: 0.554274 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'CsOPiv', 'ligand': 'PPhtBu2', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 105}
   采集函数值: -3.428596
   预测标准差: 1.470063
   与最佳点差值: 2.992969
   动态阈值: 1.470063 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'CsOAc', 'ligand': 'X-Phos', 'solvent': 'BuOAc', 'concentration': 0.153, 'temperature': 105}
   采集函数值: -2.063948
   预测标准差: 0.414784
   与最佳点差值: 1.628320
   动态阈值: 0.414784 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'KOPiv', 'ligand': 'GorlosPhos HBF4', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 90}
   采集函数值: -2.962153
   预测标准差: 1.643078
   与最佳点差值: 2.526526
   动态阈值: 1.643078 (基于方差)
   是否接受: ❌


/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3


   LLM 推荐点: {'base': 'CsOAc', 'ligand': 'PCy3 HBF4', 'solvent': 'p-Xylene', 'concentration': 0.153, 'temperature': 120}
   采集函数值: -3.849614
   预测标准差: 1.570531
   与最佳点差值: 3.413987
   动态阈值: 1.570531 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'KOAc', 'ligand': 'PPhMe2', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105}
   采集函数值: -2.491031
   预测标准差: 0.942870
   与最佳点差值: 2.055404
   动态阈值: 0.942870 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'KOPiv', 'ligand': 'tBPh-CPhos', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 105}
   采集函数值: -7.078001
   预测标准差: 0.496936
   与最佳点差值: 6.642374
   动态阈值: 0.496936 (基于方差)
   是否接受: ❌


/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

[INFO 11-14 02:1


   GP 补充推荐点: {'concentration': 0.057, 'temperature': 90, 'base': 'KOAc', 'ligand': 'X-Phos', 'solvent': 'BuCN'}

   GP 补充推荐点: {'concentration': 0.057, 'temperature': 105, 'base': 'KOPiv', 'ligand': 'P(fur)3', 'solvent': 'DMAc'}

   GP 补充推荐点: {'concentration': 0.057, 'temperature': 120, 'base': 'KOPiv', 'ligand': 'X-Phos', 'solvent': 'BuCN'}

   GP 补充推荐点: {'concentration': 0.153, 'temperature': 90, 'base': 'KOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc'}

   GP 补充推荐点: {'concentration': 0.057, 'temperature': 90, 'base': 'KOAc', 'ligand': 'P(fur)3', 'solvent': 'BuCN'}

✅ 融合推荐完成:
   LLM 推荐数: 10
   接受数: 5
   最终推荐数: 5
✅ 成功生成 5 个融合建议


In [9]:
print(f"\n📊 生成的优化建议 ({len(suggestions)} 个):")
print("=" * 80)
for i, suggestion in enumerate(suggestions, 1):
    print(f"\n建议 {i}:")
    for param_name, param_value in suggestion.items():
        print(f"  {param_name}: {param_value}")
        # 如果 param_name 是主要参数，查找实验结果
        if param_name == list(suggestion.keys())[-1]:
            # 查找是否在数据集中有匹配的实验结果
            mask = (
                (df["base"] == suggestion.get("base")) &
                (df["ligand"] == suggestion.get("ligand")) &
                (df["solvent"] == suggestion.get("solvent")) &
                (df["concentration"] == suggestion.get("concentration")) &
                (df["temperature"] == suggestion.get("temperature"))
            )
            matching_rows = df[mask]
            if len(matching_rows) > 0:
                row = matching_rows.iloc[0]
                print(f"    ▶ 实验结果: 产率={row['yield']:.2f}%, 成本={row['cost']:.4f}")
            else:
                print("    ▶ 无匹配的实验结果（新组合）")


📊 生成的优化建议 (5 个):

建议 1:
  concentration: 0.057
  temperature: 90
  base: KOAc
  ligand: X-Phos
  solvent: BuCN
    ▶ 实验结果: 产率=18.72%, 成本=0.0753

建议 2:
  concentration: 0.057
  temperature: 105
  base: KOPiv
  ligand: P(fur)3
  solvent: DMAc
    ▶ 实验结果: 产率=64.12%, 成本=0.0478

建议 3:
  concentration: 0.057
  temperature: 120
  base: KOPiv
  ligand: X-Phos
  solvent: BuCN
    ▶ 实验结果: 产率=27.80%, 成本=0.0743

建议 4:
  concentration: 0.153
  temperature: 90
  base: KOAc
  ligand: X-Phos
  solvent: DMAc
    ▶ 实验结果: 产率=38.79%, 成本=0.0300

建议 5:
  concentration: 0.057
  temperature: 90
  base: KOAc
  ligand: P(fur)3
  solvent: BuCN
    ▶ 实验结果: 产率=2.62%, 成本=0.0793


### 多轮测试

In [10]:
csv_path = "test/1728_BMS_experiments_yield_and_cost.csv"
df_multi = load_chemical_data(csv_path)
print(f"✅ 数据集加载成功: {len(df_multi)} 个实验条件")

# 构建参数空间
llm_parameters = build_llm_parameter_space(df_multi)
gp_search_space = build_gp_parameter_space(df_multi)
print(f"✅ LLM 参数空间定义完成: {len(llm_parameters)} 个参数")
print(f"✅ GP 参数空间定义完成: {len(gp_search_space)} 个参数")

# 定义优化目标
objectives = {
    "yield": {"minimize": False},  # 最大化产率
    "cost": {"minimize": True}      # 最小化成本
}
optimization_config = {
    "objectives": objectives
}
print("✅ 优化目标定义完成")
print(f"   - yield: 最大化")
print(f"   - cost: 最小化")

# 定义问题背景
problem_context = ProblemContext(
    problem_description="优化 Pd 催化的 C-H 芳基化反应条件，反应物为 N1-甲基-1H-咪唑-4-甲腈和 1-溴-2-氟苯",
    industry="化学合成 - 有机反应优化",
    domain_knowledge="""
    这是一个 Pd 催化的 C-H 芳基化反应优化问题。
    - 碱基（base）影响反应活性和选择性
    - 配体（ligand）对催化剂活性和选择性至关重要
    - 溶剂（solvent）影响反应速率和产物选择性
    - 浓度（concentration）影响反应速率和副反应
    - 温度（temperature）影响反应速率和选择性
    优化目标是同时最大化产率（yield）和最小化成本（cost）。
    """,
    constraints=[
        "所有推荐的参数组合必须符合基本化学常识"
    ],
    optimization_goals=[
        "最大化产率（yield）",
        "最小化成本（cost）"
    ]
)
print("✅ 问题背景定义完成")

# 多轮测试配置
n_rounds = 10
init_batch = 10
update_batch = 3
api_url = "http://localhost:3320"


print("\n✅ 多轮测试参数初始化完成")
print(f"   总轮次: {n_rounds}")
print(f"   初始批次: {init_batch}")
print(f"   更新批次: {update_batch}")

✅ 数据集加载成功: 1728 个实验条件
✅ LLM 参数空间定义完成: 5 个参数
✅ GP 参数空间定义完成: 5 个参数
✅ 优化目标定义完成
   - yield: 最大化
   - cost: 最小化
✅ 问题背景定义完成

✅ 多轮测试参数初始化完成
   总轮次: 10
   初始批次: 10
   更新批次: 3


In [11]:
def run_hybrid_optimization(df, llm_parameters, gp_search_space, optimization_config, 
                            problem_context, llm_config, n_rounds=10, init_batch=10, 
                            update_batch=3, api_url="http://localhost:3320", seed=None):
    """使用 HybridOptimizer 进行多轮混合优化实验"""
    
    print(f"\n🔄 开始 {n_rounds} 轮混合优化循环")
    print(f"📊 初始批次: {init_batch}, 更新批次: {update_batch}")
    
    all_experiments = []
    optimization_history = []
    
    # 第一轮：初始化（使用 init 接口）
    print(f"\n{'='*60}")
    print(f"=== 第 1 轮：初始化 ===")
    print(f"{'='*60}")
    
    init_result = call_init_api(llm_parameters, optimization_config["objectives"], 
                                batch_size=init_batch, seed=seed, api_url=api_url)
    
    if not init_result:
        print("❌ 初始化失败，退出优化循环")
        return None, None
    
    # 模拟第一轮实验结果
    round_results = simulate_experiment_results(init_result['results'], df, random_seed=seed)
    round_experiments = convert_experiment_results_to_experiment_results(round_results)
    all_experiments.extend(round_experiments)
    
    # 记录历史
    optimization_history.append({
        'round': 1,
        'type': 'init',
        'parameters': init_result['results'],
        'results': round_results,
        'best_yield': max([r['metrics']['yield'] for r in round_results]),
        'best_cost': min([r['metrics']['cost'] for r in round_results])
    })
    
    print(f"📈 第 1 轮最佳结果: yield={optimization_history[-1]['best_yield']:.2f}%, cost={optimization_history[-1]['best_cost']:.4f}")
    
    # 创建 HybridOptimizer（使用第一轮的数据）
    print(f"\n🤖 初始化 Hybrid Optimizer...")
    hybrid_optimizer = HybridOptimizer(
        llm_parameters=llm_parameters,
        gp_search_space=gp_search_space,
        gp_random_seed=seed,
        optimization_config=optimization_config,
        problem_context=problem_context,
        llm_config=llm_config,
        acquisition_threshold=0.1,
        use_dynamic_threshold=True,
        threshold_multiplier=1.0,
        random_seed=seed
    )
    
    # 添加第一轮的先验数据
    hybrid_optimizer.add_prior_experiments(round_experiments)
    print(f"✅ Hybrid Optimizer 初始化完成，已添加 {len(round_experiments)} 个先验实验")
    
    # 后续轮次：每轮重新创建 HybridOptimizer，用累积的历史数据初始化
    for round_num in range(2, n_rounds + 1):
        print(f"\n{'='*60}")
        print(f"=== 第 {round_num} 轮：混合优化 ===")
        print(f"{'='*60}")
        
        # 显示当前先验数据数量
        print(f"📊 当前累积实验数据数量: {len(all_experiments)} 个")
        
        # 每轮重新创建 HybridOptimizer，用累积的历史数据初始化
        print(f"🔄 重新创建 HybridOptimizer，使用累积的 {len(all_experiments)} 个历史实验结果...")
        hybrid_optimizer = HybridOptimizer(
            llm_parameters=llm_parameters,
            gp_search_space=gp_search_space,
            optimization_config=optimization_config,
            problem_context=problem_context,
            llm_config=llm_config,
            acquisition_threshold=0.1,
            use_dynamic_threshold=True,
            threshold_multiplier=1.0,
            random_seed=seed
        )
        
        # 添加所有累积的历史实验数据
        hybrid_optimizer.add_prior_experiments(all_experiments)
        print(f"✅ 新 HybridOptimizer 已初始化，当前先验数据数量: {len(all_experiments)} 个")
        
        # 使用新 HybridOptimizer 生成推荐
        print(f"\n🤖 使用新 HybridOptimizer 生成 {update_batch} 个优化建议...")
        suggestions = hybrid_optimizer.suggest_parameters(
            num_suggestions=update_batch,
            use_llm=True,
            use_gp=True,
            print_details=True
        )
        
        if not suggestions:
            print(f"⚠️ 第 {round_num} 轮未能生成有效建议，跳过")
            continue
        
        print(f"✅ 成功生成 {len(suggestions)} 个建议")
        
        # 显示生成的建议
        print(f"\n📋 第 {round_num} 轮生成的建议:")
        for i, suggestion in enumerate(suggestions, 1):
            print(f"\n   建议 {i}:")
            for param_name, param_value in suggestion.items():
                print(f"     {param_name}: {param_value}")
            
            # 查找是否在数据集中有匹配的实验结果
            mask = (
                (df["base"] == suggestion.get("base")) &
                (df["ligand"] == suggestion.get("ligand")) &
                (df["solvent"] == suggestion.get("solvent")) &
                (df["concentration"] == suggestion.get("concentration")) &
                (df["temperature"] == suggestion.get("temperature"))
            )
            matching_rows = df[mask]
            if len(matching_rows) > 0:
                row = matching_rows.iloc[0]
                print(f"     ▶ 实验结果: 产率={row['yield']:.2f}%, 成本={row['cost']:.4f}")
            else:
                print(f"     ▶ 无匹配的实验结果（新组合）")
        
        # 模拟实验结果
        round_results = simulate_experiment_results(suggestions, df, random_seed=seed)
        round_experiments = convert_experiment_results_to_experiment_results(round_results)
        
        # 将本轮结果加入到先验数据中
        print(f"\n📝 准备将第 {round_num} 轮的 {len(round_experiments)} 个实验结果加入到先验数据中")
        for i, new_exp in enumerate(round_experiments, 1):
            # 检查是否已经存在相同的参数组合
            is_duplicate = False
            for existing_exp in all_experiments:
                if all(existing_exp.parameters.get(k) == new_exp.parameters.get(k) 
                       for k in new_exp.parameters.keys()):
                    is_duplicate = True
                    print(f"   ⚠️ 第 {round_num} 轮建议 {i} 的参数组合已存在于先验数据中，跳过重复添加")
                    break
            if not is_duplicate:
                all_experiments.append(new_exp)
                print(f"   ✅ 第 {round_num} 轮建议 {i} 已加入先验数据")
        
        # 注意：不需要在这里更新 HybridOptimizer，因
        # 为下一轮会在生成建议之前更新
        # 这样可以确保每轮都使用累积的所有历史数据
        print(f"📊 更新后的先验数据总数: {len(all_experiments)} 个")
        
        # 记录历史
        optimization_history.append({
            'round': round_num,
            'type': 'hybrid',
            'parameters': suggestions,
            'results': round_results,
            'best_yield': max([r['metrics']['yield'] for r in round_results]),
            'best_cost': min([r['metrics']['cost'] for r in round_results])
        })
        
        print(f"\n📈 第 {round_num} 轮最佳结果: yield={optimization_history[-1]['best_yield']:.2f}%, cost={optimization_history[-1]['best_cost']:.4f}")
        
        # 显示历史最佳结果
        all_best_yields = [h['best_yield'] for h in optimization_history]
        all_best_costs = [h['best_cost'] for h in optimization_history]
        overall_best_yield = max(all_best_yields)
        overall_best_cost = min(all_best_costs)
        print(f"📊 历史最佳结果: yield={overall_best_yield:.2f}%, cost={overall_best_cost:.4f}")
        
        # 添加延迟，避免请求过快
        time.sleep(0.5)
    
    # 显示最终结果
    print(f"\n{'='*60}")
    print(f"✅ 多轮优化完成！")
    print(f"{'='*60}")
    print(f"总轮数: {len(optimization_history)}")
    print(f"总实验数: {len(all_experiments)}")
    print(f"最终最佳产率: {overall_best_yield:.2f}%")
    print(f"最终最佳成本: {overall_best_cost:.4f}")
    
    return optimization_history, all_experiments

print("✅ 多轮混合优化函数定义完成")

✅ 多轮混合优化函数定义完成


In [12]:
llm_config = LLMConfig(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.openai.com/v1"
)

hybrid_history, hybrid_experiments = run_hybrid_optimization(
    df_multi, llm_parameters, gp_search_space, optimization_config, 
    problem_context, llm_config,
    n_rounds=n_rounds,
    init_batch=init_batch,
    update_batch=update_batch,
    api_url=api_url,
    seed=42
    
)


[WARNING 11-14 02:12:01] ax.service.ax_client: Random seed set to 42. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[WARNING 11-14 02:12:01] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.
/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "base". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packag


🔄 开始 10 轮混合优化循环
📊 初始批次: 10, 更新批次: 3

=== 第 1 轮：初始化 ===
🚀 调用 init 接口，批次大小: 10
✅ Init 接口调用成功
📈 生成参数组合数: 10
💬 消息: 初始化成功，使用lhs采样生成10个参数组合
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PPh3', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 90} -> yield=5.80, cost=0.090
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'BrettPhos', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 90} -> yield=28.02, cost=0.199
✅ 找到完全匹配: {'base': 'CsOPiv', 'ligand': 'GorlosPhos HBF4', 'solvent': 'p-Xylene', 'concentration': 0.153, 'temperature': 105} -> yield=4.53, cost=0.122
✅ 找到完全匹配: {'base': 'KOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.057, 'temperature': 120} -> yield=77.58, cost=0.045
✅ 找到完全匹配: {'base': 'KOPiv', 'ligand': 'P(fur)3', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 120} -> yield=53.11, cost=0.045
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PCy3 HBF4', 'solvent': 'BuOAc', 'concentration': 0.057, 'temperature': 105} -> yield=0.00, cost=0.108
✅ 找到完全匹配: {'base': 'CsOPi

[WARNING 11-14 02:12:01] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.
/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "base". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/instantiation.py:273: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "ligand". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/service/utils/

🚀 使用GPU加速: NVIDIA A100-SXM4-80GB


[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 1 with data: {'yield': (28.02, 0.0), 'cost': (0.198802, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 2 with data: {'yield': (4.53, 0.0), 'cost': (0.121718, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 3 with data: {'yield': (77.58, 0.0), 'cost': (0.04484, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 4 with data: {'yield': (53.11, 0.0), 'cost': (0.045232, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 5 with data: {'yield': (0.0, 0.0), 'cost': (0.107935, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 6 with data: {'yield': (24.0, 0.0), 'cost': (0.155989, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 7 with data: {'yield': (24.02, 0.0), 'cost': (0.264828, 0.0)}.
[INFO 11-14 02:12:01] ax.service.ax_client: Completed trial 8 with data: {'yield': (5.02, 0.0), 'cost': (0.036398, 0.0)}.
[INFO 11-14 02:12:01] 

...


📊 GP 推荐最佳点:
   参数: {'concentration': 0.153, 'temperature': 120, 'base': 'KOPiv', 'ligand': 'PPh3', 'solvent': 'DMAc'}
   采集函数值: -0.943370
   预测标准差: 0.431171

📝 输入大模型的完整提示词:
# 优化问题背景
**问题描述**: 优化 Pd 催化的 C-H 芳基化反应条件，反应物为 N1-甲基-1H-咪唑-4-甲腈和 1-溴-2-氟苯
**行业领域**: 化学合成 - 有机反应优化
**领域知识**: 
    这是一个 Pd 催化的 C-H 芳基化反应优化问题。
    - 碱基（base）影响反应活性和选择性
    - 配体（ligand）对催化剂活性和选择性至关重要
    - 溶剂（solvent）影响反应速率和产物选择性
    - 浓度（concentration）影响反应速率和副反应
    - 温度（temperature）影响反应速率和选择性
    优化目标是同时最大化产率（yield）和最小化成本（cost）。
    
**约束条件**: 所有推荐的参数组合必须符合基本化学常识
**优化目标**: 最大化产率（yield）, 最小化成本（cost）

# 参数空间定义
以下是需要优化的参数及其范围：

**重要提示**：如果参数是离散参数，必须从列出的可选值中选择，不能选择其他值；如果参数是连续参数，值必须在 [最小值, 最大值] 范围内。
1. **base**
   描述: 碱基类型
   类型: 离散参数（必须从以下值中选择一个）
   可选值列表: [CsOAc, CsOPiv, KOAc, KOPiv]
   可选值数量: 4 个
   ⚠️ 重要：只能选择列表中的值，不能选择其他值或中间值
   值类型: str
2. **ligand**
   描述: 配体类型
   类型: 离散参数（必须从以下值中选择一个）
   可选值列表: [BrettPhos, CgMe-PPh, GorlosPhos HBF4, JackiePhos, P(fur)3, PCy3 HBF4, PPh2Me, PPh3, PPhMe2, PPhtBu2, X-Phos, tBPh-CPhos]
 

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.




   LLM 推荐点: {'base': 'CsOAc', 'ligand': 'BrettPhos', 'solvent': 'p-Xylene', 'concentration': 0.057, 'temperature': 120}
   采集函数值: -42.995394
   预测标准差: 0.312373
   与最佳点差值: 42.052024
   动态阈值: 0.312373 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'KOPiv', 'ligand': 'P(fur)3', 'solvent': 'BuOAc', 'concentration': 0.1, 'temperature': 90}
   采集函数值: -2.730013
   预测标准差: 0.340179
   与最佳点差值: 1.786643
   动态阈值: 0.340179 (基于方差)
   是否接受: ❌

   LLM 推荐点: {'base': 'KOAc', 'ligand': 'PPhMe2', 'solvent': 'BuOAc', 'concentration': 0.153, 'temperature': 90}
   采集函数值: -7.234816
   预测标准差: 0.843812
   与最佳点差值: 6.291446
   动态阈值: 0.843812 (基于方差)
   是否接受: ❌


/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.

/root/miniconda3/envs/ax_env_test/lib/python3.12/site-packages/ax/adapter/base.py:340: UserWarning:

MapKeyToFloat is unable to identify `parameters` in the transform config or an adapter with an optimization config from which the map keys can be inferred; this transform will not perform any operations and behave as a no-op.




   LLM 推荐点: {'base': 'CsOPiv', 'ligand': 'PCy3 HBF4', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 120}
   采集函数值: -42.505402
   预测标准差: 0.854872
   与最佳点差值: 41.562032
   动态阈值: 0.854872 (基于方差)
   是否接受: ❌

⚠️ GP 最佳点的采集函数值为负 (-0.943370)，说明 GP 对当前搜索空间信心不足
   采用特殊策略：直接采纳前 3 个 LLM 推荐点（按大模型推荐信心从高到低排序）

✅ 融合推荐完成:
   LLM 推荐数: 6
   接受数: 3
   最终推荐数: 3
✅ 成功生成 3 个建议

📋 第 10 轮生成的建议:

   建议 1:
     base: KOPiv
     ligand: PPh3
     solvent: BuCN
     concentration: 0.1
     temperature: 120
     ▶ 实验结果: 产率=9.59%, 成本=0.0431

   建议 2:
     base: KOAc
     ligand: X-Phos
     solvent: DMAc
     concentration: 0.153
     temperature: 105
     ▶ 实验结果: 产率=48.86%, 成本=0.0300

   建议 3:
     base: CsOAc
     ligand: BrettPhos
     solvent: p-Xylene
     concentration: 0.057
     temperature: 120
     ▶ 实验结果: 产率=38.44%, 成本=0.2055
✅ 找到完全匹配: {'base': 'KOPiv', 'ligand': 'PPh3', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 120} -> yield=9.59, cost=0.043
✅ 找到完全匹配: {'base': 'KOAc', 'ligand': 'X-

In [13]:
df = pd.DataFrame([
    {**exp.parameters, **exp.metrics, **(exp.metadata or {})}
    for exp in hybrid_experiments
])
df

,base,ligand,solvent,concentration,temperature,yield,cost
0,CsOAc,PPh3,BuCN,0.100,90,5.80,0.089659
1,CsOAc,BrettPhos,BuCN,0.153,90,28.02,0.198802
2,CsOPiv,GorlosPhos HBF4,p-Xylene,0.153,105,4.53,0.121718
3,KOAc,X-Phos,DMAc,0.057,120,77.58,0.044840
4,KOPiv,P(fur)3,BuCN,0.153,120,53.11,0.045232
5,CsOAc,PCy3 HBF4,BuOAc,0.057,105,0.00,0.107935
6,CsOPiv,CgMe-PPh,BuOAc,0.100,105,24.00,0.155989
7,KOPiv,tBPh-CPhos,p-Xylene,0.153,90,24.02,0.264828
8,KOPiv,PPh2Me,DMAc,0.057,90,5.02,0.036398
9,KOAc,PPhtBu2,DMAc,0.057,120,2.13,0.053335
